In [2]:
import pandas as pd
import numpy as np
from functools import partial

def latest_birth_year(column: str, stop: pd.Series) -> pd.Timestamp:
  if pd.isnull(stop[column]): return np.nan
  return (stop["date_time"] - pd.DateOffset(years=stop[column], days=1)).year

stops_df = pd.read_csv("../files/events/stops_full.csv", parse_dates=["date_time"], date_parser=pd.to_datetime)

stops_df["first_off_latest_birth_year"] = stops_df.apply(partial(latest_birth_year, "first_off_age"), axis=1)
stops_df["second_off_latest_birth_year"] = stops_df.apply(partial(latest_birth_year, "second_off_age"), axis=1)
stops_df.sort_values("first_off_latest_birth_year", inplace=True)


/tmp/ipykernel_19860/1810155183.py:9: DtypeWarning: Columns (3,5,6,7,8,10,11,12,13,20,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  stops_df = pd.read_csv("../files/events/stops_full.csv", parse_dates=["date_time"], date_parser=pd.to_datetime)


In [3]:
from data_tweak.converters import convert_race, convert_sex
convert_sex(stops_df, ["first_off_sex", "second_off_sex"])
convert_race(stops_df, ["first_off_race", "second_off_race"])
stops_df.index.name = "stop_id"
stops_df.to_csv("../files/events/stops_by_fo_birth_year.csv")

In [39]:
prof_df = pd.read_csv("../files/profiles/officers_index_plus_kiefer.csv")
print("number of profiles:", prof_df.shape[0])
prof_df = prof_df.dropna(subset=["off_year_of_birth"]).sort_values("off_year_of_birth")
print("number of profiles with a birth year:", prof_df.shape[0])
print("number of stops:", stops_df.shape[0])
stops_df_cleaned = stops_df.dropna(subset=["first_off_latest_birth_year"])
print("number of stops with a birth year:", stops_df_cleaned.shape[0])
cols = list(stops_df_cleaned.columns.tolist()) + ["off_uniq_id"]

pd.merge_asof(stops_df_cleaned, prof_df, left_on=["first_off_latest_birth_year"], right_on=["off_year_of_birth"], left_by=["first_off_first_name", "first_off_last_name", "first_off_sex", "first_off_race"], right_by=["off_first_name", "off_last_name", "off_sex", "off_race"], tolerance=1, direction="nearest").loc[:, cols].to_csv("../files/events/test/stops_age_merge_att.csv")


number of profiles: 36323
number of profiles with a birth year: 36321
number of stops: 2865566
number of stops with a birth year: 2258099


In [40]:
merged_fo_df = pd.read_csv("../files/events/test/stops_age_merge_att.csv")
print("number of matches", merged_fo_df[merged_fo_df["off_uniq_id"].notna()].shape[0])

/tmp/ipykernel_4287/2318788398.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_fo_df = pd.read_csv("../files/events/test/stops_age_merge_att.csv")


number of matches 2173038
